In [1]:
from util import *
from trueskill import Rating, rate, BETA, global_env, setup, quality
import itertools
import math
import pandas as pd
# 根据 Trueskill 的 rating 模型，计算胜率
def win_probability(team1, team2):
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (BETA * BETA) + sum_sigma)
    ts = global_env()
    return ts.cdf(delta_mu / denom)

In [2]:
data = load_json('data/wc_sofifa.json')
df = pd.DataFrame(data)[['name', 'overall', 'positions', 'club']]

In [3]:
elos = load_json('data/elo.json')
df['elo'] = df['club'].apply(lambda x: elos[x])
df[df['club'] == 'Iceland']

,name,overall,positions,club,elo
70,G. Sigurðsson,85,"[CM, CAM]",Iceland,1764
269,A. Finnbogason,80,[ST],Iceland,1764
373,J. Guðmundsson,79,"[RM, CAM]",Iceland,1764
489,A. Gunnarsson,78,"[CDM, CM]",Iceland,1764
557,R. Sigurðsson,77,[CB],Iceland,1764
628,E. Hallfreðsson,76,"[CM, CDM]",Iceland,1764
649,B. Bjarnason,76,"[LM, CAM, CM, RM]",Iceland,1764
769,K. Árnason,75,[CB],Iceland,1764
817,H. Halldórsson,74,[GK],Iceland,1764
843,B. Sævarsson,74,[RB],Iceland,1764


In [4]:
matchups = [{
    'Portugal': [0, 25, 75, 82, 261, 89, 110, 130, 252, 333, 177],
    'Spain': [4, 11, 31, 37, 40, 41, 43, 67, 68, 103, 161]
    },
    {
    'France': [12, 17, 38, 44, 56, 61, 62, 114, 117, 384, 417],
    'Australia': [609, 638, 825, 999, 1024, 1080, 1109, 1127, 1165, 1327, 1350]
    },
    {
    'Argentina': [1, 39, 47, 71, 210, 244, 253, 340, 358, 359, 977],
    'Iceland': [70, 269, 373, 489, 557, 628, 649, 769, 817, 843, 872]
    },
]
# 0.22 来自所有国家队比赛数据
setup(draw_probability=0.22, sigma=2000/3, beta=2000/3/2)
def calculate_with_matchup(matchup):
    teams = []
    nations = []
    for nation, indexes in matchup.items():
        ratings = []
        nations.append(nation)
#         print(nation)
        for i in indexes:
            player = df.iloc[i]
            ratings.append(Rating(mu=player['overall']+player['elo']))
#             print(player['name'], ratings[-1])
        teams.append(ratings)
    print("{0} 胜率：".format(nations[0]), win_probability(teams[0], teams[1]))
    print("{0} 胜率：".format(nations[1]),win_probability(teams[1], teams[0]))
    print("平局概率：", quality(teams))
    print()

for item in matchups:
    calculate_with_matchup(item)

Portugal 胜率： 0.41068817033
Spain 胜率： 0.58931182967
平局概率： 0.43818699966094404

France 胜率： 0.816933404522
Australia 胜率： 0.183066595478
平局概率： 0.32257156927766534

Argentina 胜率： 0.788088653558
Iceland 胜率： 0.211911346442
平局概率： 0.3462461430451184

